In [1]:
import json
import requests
import pandas as pd
import numpy as np
import folium
import pprint
import sys
import warnings
from folium.plugins import MarkerCluster


warnings.simplefilter("ignore")


with open ('city.list.json','r', encoding='utf-8') as fin:
    city_list_data = json.load(fin)
    

def city_id(city_list_data):
    city_name = input('Please Enter a City or State Name: ')
    citylist_df = pd.DataFrame(city_list_data)
    return  citylist_df[citylist_df['name'] == city_name.title()]
    
    
def city_map(city_blocks):
    CENTER_STATES =(39,-96)
    choose_city = folium.Map(location=CENTER_STATES, zoom_start=3)
    
    for index,row in city_blocks.iterrows():
        coords_list = [row['coord']['lat'],row['coord']['lon']]
        convert_id = np.str(row['id'])
        folium.Marker(coords_list, popup =convert_id).add_to(choose_city)
    
    return choose_city


def city_coord(city_blocks):
    
    city_id = int(input('Click on the location, and Enter the city ID: '))

    return city_blocks[city_blocks['id'] == city_id]['coord'].values[0]


def weatherdataframe(lat,lon):
    loc_coord = "%i,%i,%i,%i" % (lon+3,lat-3,lon-3,lat+3)
    loc = 'bbox='+loc_coord+',10'
    key = 'APPID=49e640103f17dad25ae91b080e01970b'
    url = 'http://api.openweathermap.org/data/2.5/box/city?' + loc +'&'+ key 
    
    responds = requests.get(url)
    data = responds.json()                  #gets raw data from the api
    return pd.DataFrame(data['list'])

 
    
def weather_map(weatherDF):
    CENTER_CITY = (lat,lon)
    map1 = folium.Map(location=CENTER_CITY, zoom_start=7)
    marker_cluster = folium.plugins.MarkerCluster().add_to(map1)
    
    for row in weatherDF.to_records():
        coords = (row['coord']['Lat'],row['coord']['Lon'])
        info = row['name']+' ||| Humidity: '+str(row['main']['humidity'])+'  Pressure: '+str(row['main']['pressure'])+'  Temp: '+str(row['main']['temp'])+'° C  Max Temp: '+str(row['main']['temp_max'])+'° C  Min Temp: '+str(row['main']['temp_min'])+'° C'
        color = get_color(row)
        folium.CircleMarker(location=coords, radius=15, popup=info,color=color,fill_color=color).add_to(marker_cluster)

    return map1

    
def get_color(row):
    if  row['main']['temp'] >= 35:
        circle_color = '#840319'
    elif row['main']['temp'] >= 30:
        circle_color = '#e51b1b'
    elif row['main']['temp'] >= 25:
        circle_color = '#e5791b'
    elif row['main']['temp'] >= 20:
        circle_color = '#e5b91b'
    elif row['main']['temp'] >= 15:
        circle_color = '#eae71c'
    elif row['main']['temp'] >= 10:
        circle_color = '#94ff42'
    elif row['main']['temp'] >= 5:
        circle_color = '#34f474'
    elif row['main']['temp'] >= 0:
        circle_color = '#34f4d4'
    elif row['main']['temp'] >= -5:
        circle_color = '#7ccdff'
    elif row['main']['temp'] >= -10:
        circle_color = '#7972ff'
    elif row['main']['temp'] >= -15:
        circle_color = '#3c15c6'
    else:
        circle_color = '#64259b'
    return circle_color

In [2]:
city_blocks = city_id(city_list_data)
city_map(city_blocks)

Please Enter a City or State Name: sandwich


In [3]:
try:
    city_coord = city_coord(city_blocks)
except ValueError:
    sys.exit('Entered value was not a city ID, please restart the program!')
except IndexError:
    sys.exit('Entered value was not recognized as city ID, please restart the program!')
    
lon = city_coord['lon']
lat = city_coord['lat']

weatherDF = weatherdataframe(lat,lon)
weather_map(weatherDF)

Click on the location, and Enter the city ID: 5140405
